In [13]:
#import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [14]:
####################
# GLOBAL VARIABLES #
####################
dta_pth = '../data/'
fn = 'news.csv'
f_pth = os.path.join(dta_pth,fn)
y_map = {'FAKE':0,'REAL':1}
X_col = 'title' # can be changed to 'text' to reproduce results for text of article column
y_col = 'label'
random_state = 42

showTokens = True
wordnet_lemmatizer = WordNetLemmatizer()

df = pd.read_csv(f_pth)
print('File "{}": {} rows x {} columns\n'.format(fn,df.shape[0],df.shape[1]))
df.info()

File "news.csv": 6335 rows x 4 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
Unnamed: 0    6335 non-null int64
title         6335 non-null object
text          6335 non-null object
label         6335 non-null object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [15]:
# Park this for later - try using regex to pull our text/titles that may have been preprocessed differently.
a = {i:df['text'].str.contains(r'\x{}'.format(i)).sum() for i in range(90,101)}
sum(list(a.values()))


215